In [63]:
from timeit import default_timer as timer
import numpy as np
import random
from queue import PriorityQueue
def make_board(size):
    board = [[0 for j in range(size)] for i in range(size)]
    return board
    
def place_queen(board, x, y):
    if board[x][y] != 1:
        board[x][y] = 1
        print("ADDED QUEEN AT", x, ",", y, ".")
        #print(np.matrix(board))
    else:
        print("UNABLE TO PLACE QUEEN. THERE IS ALREADY A QUEEN LOCATED HERE")

def random_queen_placement(board):
    board = [[0 for j in range(len(board))] for i in range(len(board))]
    for i in range(len(board)):
        j = random.randint(0, len(board) - 1)
        board[i][j] = 1
        
    return board

"""RETURNS ARRAY OF POSITIONS OF QUEENS ON THE BOARD"""
def get_queens(board):
    queens = []
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 1:
                queens.append([i, j])
    return queens

"""RETURNS AN ARRAY OF ALL SQUARES ATTACKED BY A SINGLE QUEEN"""
def get_attacking_squares(board, queen):
    #print("GET ATTACKING SQUARES:")
    attacks = []
    #print(queen)
    #print("VERTICAL") # y axis, first dimension
    for i in range(len(board)):
        if i != queen[0]:
            attacks.append([i, queen[1]])
    #print("HORIZONTAL") # x axis, second dimension
    for i in range(len(board)):
        if i != queen[1]:
            attacks.append([queen[0], i])
    #print("DIAGONAL")
    for i in range(1, len(board)): #down right
        if i + queen[0] >= len(board) or i + queen[1] >= len(board):
            break
        attacks.append([i + queen[0], queen[1] + i])
    for i in range(1, len(board)): #up right
        if queen[0] - i < 0 or queen[1] + i >= len(board):
            break
        attacks.append([queen[0] - i, queen[1] + i])
    for i in range(1, len(board)): #down left
        if i + queen[0] >= len(board) or queen[1] - i < 0:
            break
        attacks.append([i + queen[0], queen[1] - i])
    for i in range(1, len(board)): #up left
        if queen[0] - i < 0 or queen[1] - i < 0:
            break
        attacks.append([queen[0] - i, queen[1] - i])
    return attacks

"""DETERMINE HOW MANY PIECES(QUEENS) A QUEEN IS ATTACKING"""
def count_attacks(board, queen):
    count = 0
    a = get_attacking_squares(board, queen)
    for square in a:
        if board[square[0]][square[1]] == 1:
            #print("found queen at", square)
            count += 1
    return count

"""DETERMINE WHICH QUEEN IS ATTACKING THE MOST PIECES"""
def worst_queen(board, queens): #
    worst = [queens[0], count_attacks(board, queens[0])]
    for queen in queens:
        if count_attacks(board, queen) > worst[1]:
            worst = [queen, count_attacks(board, queen)]
    return worst[0]

def fitness(board):
    queens = get_queens(board)
    a_count = 0
    for queen in queens:
        a_count += count_attacks(board, queen)
    return a_count//2

"""THIS IS WHERE THE "HILL CLIMBING" OCCURS"""
def evaluate_move(board, queen): # choses a move for a queen
    #print("BOARD, QUEEN", board, queen)
    a = get_attacking_squares(board, queen)
    #print("ATTACKING SQUARES:", a)
    moves = []
    score = 0
    for square in a: # ATTACKING SQUARES. PICK ONE THAT IS BETTER THAN CURRENT.
        if board[square[0]][square[1]] == 1: # SKIP SQAURES THAT CONTAIN PIECES. CAN'T MOVE THERE.
            continue 
        score = count_attacks(board, square) - 1 # subract 1 becasue queen move would be attacking itself.
        moves.append([score, square])
    #print(moves)
    best_index = 0
    best_move = moves[best_index]
    for move in moves:
        if move[0] < moves[0][0]:
            best_move = move
    #print("MOVE:", queen, "->", best_move[1])
    return best_move[1]
    
def execute_move(board, queen, move):
    #print(board, queen, move)
    #print("REMOVING QUEEN")
    board[queen[0]][queen[1]] = 0
    queens = get_queens(board)
    #print("ADDING QUEEN")
    board[move[0]][move[1]] = 1
    queens = get_queens(board)
    return board, queens

def get_successor(board, queens):
    w = worst_queen(board, queens)
    best = evaluate_move(board, w)
    board, queens = execute_move(board, w, best)
    #print(np.matrix(board))
    #print("FITNESS", fitness(board))
    return board, queens

def attempt_n_queens(n): #TODO: CHECK IF FITNESS DECREASED
    game = make_board(n)
    game = random_queen_placement(game)
    #print("INITIAL SET UP:", "FITNESS", fitness(game))
    #print(np.matrix(game))
    moves = 0
    while fitness(game) != 0 and moves < 100:
        game, queens = get_successor(game, get_queens(game))
        moves += 1
    if fitness(game) == 0:
        print("WINNER!")
        print(np.matrix(game))
        return True
    return False

# print("ATTEMPT N QUEENS:")
# WINNER = False
# count = 0
# while not WINNER:
#     WINNER = attempt_n_queens(20)
#     count += 1
#     if count % 100 == 0:
#         print(count)
# print(count, "ATTEMPTS")

# for i in range(4, 20):
#     winner = False
#     print("Attempting", i, "Queens")
#     while not winner:
#         winner = attempt_n_queens(i)

if __name__ == '__main__':
    start = timer()
    winner = False
    restarts = 0
    while not winner:
        winner = attempt_n_queens(5)
        restarts += 1
    end = timer()
    formatted_time = "{:.2f}".format((end - start) * 1000)
    print("running time:", formatted_time, "ms")
    print("restarts:", restarts)
        


WINNER!
[[0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]]
running time: 2.85 ms
restarts: 1


In [72]:
for i in range(13, 20):
    winner = False
    print("Attempting", i, "Queens")
    while not winner:
        winner = attempt_n_queens(i)

Attempting 13 Queens
WINNER!
[[0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0]]
Attempting 14 Queens


KeyboardInterrupt: 